In [51]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import Select
import time
import pandas as pd
import re

In [45]:
pd.set_option('display.max_rows',None)


In [46]:
pd.set_option('display.max_colwidth', None)

In [63]:
options = Options() 
driver = webdriver.Chrome(options=options)
driver.get('https://www.clasificadosonline.com/RealEstate.asp')

In [49]:
multi_check_box_toggle = driver.find_element(By.XPATH,'//*[@id="form1"]/div/table[2]/tbody/tr/td/div[1]')
multi_check_box_toggle.click()
time.sleep(2)

In [50]:
select_all = driver.find_element(By.XPATH,'//*[@id="form1"]/div/table[2]/tbody/tr/td/div[2]/div[1]/input')
select_all.click()
time.sleep(2)

In [51]:
time.sleep(10)
ver_listado = driver.find_element(By.XPATH,'//*[@id="BtnSearchListing"]')
ver_listado.click()

In [54]:
order_by = driver.find_element(By.XPATH,'//*[@id="jumpMenu"]')

In [ ]:
order_by = driver.find_element(By.XPATH,'//*[@id="jumpMenu"]')
order_by.click()
ultimos_publicados = driver.find_element(By.XPATH,'//*[@id="jumpMenu"]/option[7]')
ultimos_publicados.click()

In [12]:
list_cards = []

for i in range(1,16):
    table_1 = '#listing > table > tbody > tr > td > table > tbody > tr:nth-child(2) > td > div > div:nth-child(2) > table:nth-child(1)'
    table_1_title = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(1) > td > a > div > span'
    table_1_rooms = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(1) > span'
    table_1_price = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(2) > span:nth-child(2) > font'
    table_1_type = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(2) > span:nth-child(4)'
    table_1_location = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(3) > td'
    table_1_link = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(1) > td > a'
    
    table_2 = '#listing > table > tbody > tr > td > table > tbody > tr:nth-child(2) > td > div > div:nth-child(2) > table:nth-child(3)'
    table_2_title = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(1) > td > a > div > span'
    table_2_rooms = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(1) > span'
    table_2_price = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(2) > span:nth-child(2) > font'
    table_2_type = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(2) > span:nth-child(4)'
    table_2_location = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(3) > td'
    table_2_link = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(1) > td > a'
    
    dictionary_cards = {}
    dictionary_cards_2 = {}
    try:
        table_1_type_2 = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(2) > span:nth-child(5)'
        table_1_type_2_text = driver.find_element(By.CSS_SELECTOR, table_1_type_2).text
        dictionary_cards['Type'] = driver.find_element(By.CSS_SELECTOR, table_1_type).text + ' ' + table_1_type_2_text
        
        table_2_type_2 = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(2) > span:nth-child(5)'
        table_2_type_2_text = driver.find_element(By.CSS_SELECTOR, table_2_type_2).text
        dictionary_cards_2['Type'] = driver.find_element(By.CSS_SELECTOR, table_2_type).text + ' ' + table_2_type_2_text

    except NoSuchElementException:
        dictionary_cards['Type'] = driver.find_element(By.CSS_SELECTOR, table_1_type).text
        dictionary_cards_2['Type'] = driver.find_element(By.CSS_SELECTOR, table_2_type).text
    try:
        table_1_broker = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(2) > center > a > img'
        dictionary_cards['Broker'] = driver.find_element(By.CSS_SELECTOR, table_1_broker).get_attribute('alt')

        table_2_broker = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(2) > center > a > img'
        dictionary_cards_2['Broker'] = driver.find_element(By.CSS_SELECTOR, table_2_broker).get_attribute('alt')

    except NoSuchElementException:
        dictionary_cards['Broker'] = 'No Broker'
        dictionary_cards_2['Broker'] = 'No Broker'

    dictionary_cards['Title'] = driver.find_element(By.CSS_SELECTOR, table_1_title).text
    dictionary_cards['Rooms'] = driver.find_element(By.CSS_SELECTOR, table_1_rooms).text
    dictionary_cards['Price'] = driver.find_element(By.CSS_SELECTOR, table_1_price).text
    dictionary_cards['Location'] = driver.find_element(By.CSS_SELECTOR, table_1_location).text
    dictionary_cards['Link'] = driver.find_element(By.CSS_SELECTOR, table_1_link).get_attribute('href')
    list_cards.append(dictionary_cards)

    dictionary_cards_2['Title'] = driver.find_element(By.CSS_SELECTOR, table_2_title).text
    dictionary_cards_2['Rooms'] = driver.find_element(By.CSS_SELECTOR, table_2_rooms).text
    dictionary_cards_2['Price'] = driver.find_element(By.CSS_SELECTOR, table_2_price).text
    dictionary_cards_2['Location'] = driver.find_element(By.CSS_SELECTOR, table_2_location).text
    dictionary_cards_2['Link'] = driver.find_element(By.CSS_SELECTOR, table_2_link).get_attribute('href')
    list_cards.append(dictionary_cards_2)


In [ ]:
df = pd.json_normalize(list_cards)
df

In [ ]:
next_page = driver.find_element(By.XPATH,'//*[@id="listing"]/table/tbody/tr/td/table/tbody/tr[2]/td/table[5]/tbody/tr[1]/td[3]/div/a')
next_page.click()

In [5]:
last_modified = driver.execute_script("return document.lastModified;")

In [ ]:
last_modified

In [ ]:
import requests

# Send a GET request to the webpage
response = requests.get("https://www.clasificadosonline.com/UDRealEstateDetail.asp?ID=4774562")  # Replace this URL with the URL of your webpage

# Check the Last-Modified header in the response
last_modified_header = response.headers.get("Last-Modified")

# Print the Last-Modified header
print("Last modified (from HTTP header):", last_modified_header)

In [ ]:
all_cards = []

# Infinite loop to navigate through pages
for x in range(3):
    # Extract data from the current page
    list_cards = []

    for i in range(1, 16):  # Adjust the range as per the number of rows on a page
        dictionary_cards = {}
        dictionary_cards_2 = {}
        
        # Table 1 Data
        table_1 = '#listing > table > tbody > tr > td > table > tbody > tr:nth-child(2) > td > div > div:nth-child(2) > table:nth-child(1)'
        table_1_title = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(1) > td > a > div > span'
        table_1_rooms = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(1) > span'
        table_1_price = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(2) > span:nth-child(2) > font'
        table_1_type = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(2) > span:nth-child(4)'
        table_1_location = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(3) > td'
        table_1_link = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(1) > td > a'

        try:
            dictionary_cards['Title'] = driver.find_element(By.CSS_SELECTOR, table_1_title).text
            dictionary_cards['Rooms'] = driver.find_element(By.CSS_SELECTOR, table_1_rooms).text
            dictionary_cards['Price'] = driver.find_element(By.CSS_SELECTOR, table_1_price).text
            dictionary_cards['Type'] = driver.find_element(By.CSS_SELECTOR, table_1_type).text
            dictionary_cards['Location'] = driver.find_element(By.CSS_SELECTOR, table_1_location).text
            dictionary_cards['Link'] = driver.find_element(By.CSS_SELECTOR, table_1_link).get_attribute('href')
            list_cards.append(dictionary_cards)
        except NoSuchElementException:
            pass

        # Table 2 Data (similar logic for table_2)
        table_2 = '#listing > table > tbody > tr > td > table > tbody > tr:nth-child(2) > td > div > div:nth-child(2) > table:nth-child(3)'
        table_2_title = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(1) > td > a > div > span'
        table_2_rooms = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(1) > span'
        table_2_price = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(2) > span:nth-child(2) > font'
        table_2_type = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(2) > span:nth-child(4)'
        table_2_location = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(3) > td'
        table_2_link = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(1) > td > a'

        try:
            dictionary_cards_2['Title'] = driver.find_element(By.CSS_SELECTOR, table_2_title).text
            dictionary_cards_2['Rooms'] = driver.find_element(By.CSS_SELECTOR, table_2_rooms).text
            dictionary_cards_2['Price'] = driver.find_element(By.CSS_SELECTOR, table_2_price).text
            dictionary_cards_2['Type'] = driver.find_element(By.CSS_SELECTOR, table_2_type).text
            dictionary_cards_2['Location'] = driver.find_element(By.CSS_SELECTOR, table_2_location).text
            dictionary_cards_2['Link'] = driver.find_element(By.CSS_SELECTOR, table_2_link).get_attribute('href')
            list_cards.append(dictionary_cards_2)
        except NoSuchElementException:
            pass

    # Append the current page's data to the main list
    all_cards.extend(list_cards)

    # Try to click the "Next" button
    try:
        time.sleep(10)
        next_page = driver.find_element(By.XPATH, '//*[@id="listing"]/table/tbody/tr/td/table/tbody/tr[2]/td/table[5]/tbody/tr[1]/td[3]/div/a')
        next_page.click()
        time.sleep(7)  # Add delay to allow page to load
    except NoSuchElementException:
        print("No more pages available.")
        break

# Create a single DataFrame from all collected data
df = pd.DataFrame(all_cards)

# Save to CSV or process further
df.to_csv("classifieds_data.csv", index=False)

print("Data collection completed.")

In [61]:
def extract_property_id(link):
    match = re.search(r"ID=(\d+)", link)
    if match:
        return match.group(1)
    return None

In [96]:
def clean_real_estate_data(df):
    # Clean the 'Type' column by removing leading commas and whitespace
    df['type'] = df['type'].str.replace(r'^,\s*', '', regex=True).str.strip()
    
    # Clean the 'Broker' column using the combined logic
    def clean_broker(broker_name):
        try:
            # Remove any text starting with " - " followed by anything after it
            broker_name = re.sub(r'\s*-\s*.*$', '', broker_name)
            
            # Remove any text starting with "#" followed by digits
            broker_name = re.sub(r'\s*#\d+', '', broker_name)
            
            # Remove any text starting with "lic." followed by digits
            broker_name = re.sub(r'\s*lic\.\s*\d+', '', broker_name)
            
        except Exception as e:
            print(f"An error occurred: {e}")

        # Return the cleaned broker name
        return broker_name.strip()
    
    # Apply the clean_broker function to the 'Broker' column and remove the "ClasificadosOnline" text
    df['broker'] = df['broker'].apply(clean_broker)
    df['broker'] = df['broker'].str.replace('ClasificadosOnline', '').str.strip()

    # Extract the number of bedrooms and bathrooms from the 'Rooms' column
    # Extract bedrooms (including cases with >= notation)
    df['bedrooms'] = df['rooms'].str.extract(r'(>=\s*\d+|\d+)\s+Cuartos', expand=False)

    # Extract bathrooms
    df['bathrooms'] = df['rooms'].str.extract(r'(>=\s*\d+|\d+)\s+Baños', expand=False)

    # Fill NaN values with empty strings (or another placeholder if needed)
    df[['bedrooms', 'bathrooms']] = df[['bedrooms', 'bathrooms']].fillna("0")

    # Fill missing values with 0 (as strings to maintain consistency)
    #df[['bedrooms', 'bathrooms']] = df[['bedrooms', 'bathrooms']].fillna('0')

    # Drop the Rooms column
    df.drop(columns=['rooms'], inplace=True)

    # Fill missing values in the 'barrio' and 'pueblo' columns with empty strings
    df['barrio'] = df['barrio'].fillna('')
    df['pueblo'] = df['pueblo'].fillna('')

    # Strip any leading or trailing whitespace from the 'barrio' and 'pueblo' columns
    df['barrio'] = df['barrio'].str.strip()
    df['pueblo'] = df['pueblo'].str.strip()

    # Load the municipio data from the CSV file
    municipio = pd.read_csv('PR_Municipios')

    # Check if the word from the name column in municipio is present in the pueblo column within data
    df['pueblo_name'] = df['pueblo'].apply(lambda x: next((name for name in municipio['name'] if name in x), None))

    # Merge the dataframes on the extracted pueblo names and the name column
    merged_df = pd.merge(df, municipio, left_on='pueblo_name', right_on='name', how='left')

    # Add the region column to the original data dataframe
    df['region'] = merged_df['region']

    # Drop the temporary pueblo_name column
    df.drop(columns=['pueblo_name'], inplace=True)

    # Clean the 'price' column by removing the dollar sign and commas and converting it to a float
    df['price'] = df['price'].str.replace('$','').str.replace(',','').astype(float)

    # Remove duplicate rows based on the 'PropertyID' column
    # Keep the first occurrence and log the duplicates
    duplicates = df[df.duplicated(subset=['propertyid'], keep=False)]
    if not duplicates.empty:
        print("Duplicate Properties Found:")
        print(duplicates)

    # Drop duplicates and keep the first occurrence
    df = df.drop_duplicates(subset=['propertyid'], keep='first').reset_index(drop=True)

    return df

In [97]:
# Initialize an empty list to store all data
all_cards = []

# Initialize a page counter (optional, for debugging or tracking pages)
page_number = 1

# Infinite loop to navigate through pages
for x in range(1):

    dropdown_element = driver.find_element(By.XPATH, '//*[@id="jumpMenu"]')
    select = Select(dropdown_element)
    selected_option = select.first_selected_option

    print(f"Scraping page {page_number}...")

    if selected_option.text !=  'Ultimos Publicados (Más Recientes Primeros)':
        dropdown_element.click()
        time.sleep(2)
        ultimos_publicados = driver.find_element(By.XPATH,'//*[@id="jumpMenu"]/option[7]')
        ultimos_publicados.click()
        time.sleep(7)


    # Extract data from the current page
    list_cards = []

    for i in range(1, 16):  # Adjust the range as per the number of rows on a page
        dictionary_cards = {}
        dictionary_cards_2 = {}
        
        # Table 1 Data
    
        table_1 = '#listing > table > tbody > tr > td > table > tbody > tr:nth-child(2) > td > div > div:nth-child(2) > table:nth-child(1)'
        table_1_title = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(1) > td > a > div > span'
        table_1_rooms = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(1) > span'
        table_1_price = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(2) > span:nth-child(2) > font'
        table_1_type = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(2) > span:nth-child(4)'
        table_1_type_2 = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(2) > span:nth-child(5)'
        table_1_barrio = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(3) > td > a:nth-child(1)'
        table_1_pueblo = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(3) > td > a:nth-child(3)'
        table_1_link = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(1) > td > a'
        table_1_broker = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(2) > center > a > img'
        table_1_piclink = table_1 + f'> tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(1) > table > tbody > tr > td > div > a > img'
        
        try:
            dictionary_cards['title'] = driver.find_element(By.CSS_SELECTOR, table_1_title).text
            dictionary_cards['rooms'] = driver.find_element(By.CSS_SELECTOR, table_1_rooms).text
            dictionary_cards['price'] = driver.find_element(By.CSS_SELECTOR, table_1_price).text
            
            # type validation, since a property can have two types
            try: 

                table_1_type_2_text = driver.find_element(By.CSS_SELECTOR, table_1_type_2).text
                dictionary_cards['type'] = driver.find_element(By.CSS_SELECTOR, table_1_type).text + ' ' + table_1_type_2_text

            except NoSuchElementException:
                dictionary_cards['type'] = driver.find_element(By.CSS_SELECTOR, table_1_type).text


            dictionary_cards['barrio'] = driver.find_element(By.CSS_SELECTOR, table_1_barrio).text
            dictionary_cards['pueblo'] = driver.find_element(By.CSS_SELECTOR, table_1_pueblo).text
            link_1 = driver.find_element(By.CSS_SELECTOR, table_1_link).get_attribute('href')
            dictionary_cards['link'] = link_1
            dictionary_cards['propertyid'] = extract_property_id(link_1)

            if "MultipleSellers" in dictionary_cards['link']:
                dictionary_cards['broker'] = 'Multiple Sellers'

            else:

                # not all properties have brokers
                try:
                    dictionary_cards['broker'] = driver.find_element(By.CSS_SELECTOR, table_1_broker).get_attribute('alt')
                except NoSuchElementException:
                    dictionary_cards['broker'] = 'No Broker'


            dictionary_cards['piclink'] = driver.find_element(By.CSS_SELECTOR, table_1_piclink).get_attribute('src')

            list_cards.append(dictionary_cards)
            
        except NoSuchElementException:
            pass

        # Table 2 Data (similar logic for table_2)
        table_2 = '#listing > table > tbody > tr > td > table > tbody > tr:nth-child(2) > td > div > div:nth-child(2) > table:nth-child(3)'
        table_2_title = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(1) > td > a > div > span'
        table_2_rooms = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(1) > span'
        table_2_price = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(2) > span:nth-child(2) > font'
        table_2_type = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(2) > span:nth-child(4)'
        table_2_type_2 = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(2) > span:nth-child(5)'
        table_2_barrio = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(3) > td > a:nth-child(1)'
        table_2_pueblo = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(3) > td > a:nth-child(3)'
        table_2_link = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(1) > td > a'
        table_2_broker = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(2) > center > a > img'
        table_2_piclink = table_2 + f'> tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(1) > table > tbody > tr > td > div > a > img'

        try:
            dictionary_cards_2['title'] = driver.find_element(By.CSS_SELECTOR, table_2_title).text
            dictionary_cards_2['rooms'] = driver.find_element(By.CSS_SELECTOR, table_2_rooms).text
            dictionary_cards_2['price'] = driver.find_element(By.CSS_SELECTOR, table_2_price).text
            
            # type validation, since a property can have two types
            try:
                table_2_type_2_text = driver.find_element(By.CSS_SELECTOR, table_2_type_2).text
                dictionary_cards_2['type'] = driver.find_element(By.CSS_SELECTOR, table_2_type).text + ' ' + table_2_type_2_text
            except NoSuchElementException:
                dictionary_cards_2['type'] = driver.find_element(By.CSS_SELECTOR, table_2_type).text
            
            dictionary_cards_2['barrio'] = driver.find_element(By.CSS_SELECTOR, table_2_barrio).text
            dictionary_cards_2['pueblo'] = driver.find_element(By.CSS_SELECTOR, table_2_pueblo).text
            link_2 = driver.find_element(By.CSS_SELECTOR, table_2_link).get_attribute('href')
            dictionary_cards_2['link'] = link_2
            dictionary_cards_2['propertyid'] = extract_property_id(link_2)

            if "MultipleSellers" in dictionary_cards_2['link']:
                dictionary_cards_2['broker'] = 'Multiple Sellers'

            else:
                try:
                    dictionary_cards_2['broker'] = driver.find_element(By.CSS_SELECTOR, table_2_broker).get_attribute('alt')
                except NoSuchElementException:
                    dictionary_cards_2['broker'] = 'No Broker'

            dictionary_cards_2['piclink'] = driver.find_element(By.CSS_SELECTOR, table_2_piclink).get_attribute('src')

            list_cards.append(dictionary_cards_2)
            
        except NoSuchElementException:
            pass

    # Append the current page's data to the main list
    all_cards.extend(list_cards)

    # Try to click the "Next" button
    try:
        #time.sleep(10)
        #next_page = driver.find_element(By.XPATH, '//*[@id="listing"]/table/tbody/tr/td/table/tbody/tr[2]/td/table[5]/tbody/tr[1]/td[3]/div/a')
        #next_page.click()
        #time.sleep(7)  # Add delay to allow page to load
        page_number += 1
    except NoSuchElementException:
        print("No more pages available.")
        break

# Create a single DataFrame from all collected data
df = pd.DataFrame(all_cards)
cleaned_df = clean_real_estate_data(df)

Scraping page 1...


In [ ]:
cleaned_df = clean_real_estate_data(df)

In [30]:
cleaned_df.to_csv("classifieds_data_v5.csv", index=False)

In [2]:
data = pd.read_csv('classifieds_data_v3.csv')

In [57]:
cleaned_df['type'].unique()

array(['Apt/WalkUp', 'Casa', 'Comercial', 'Finca', 'Apartamento', 'Solar',
       'MultiFamiliar'], dtype=object)

In [60]:
cleaned_df[cleaned_df['broker'] == 'No Broker']

,title,rooms,price,type,barrio,pueblo,location,link,propertyid,broker,piclink,bedrooms,bathrooms,neighborhood,city,region
35,Condo. NEXEL en zona histórica de Pon...,3 Cuartos | 1 Baños,125000.0,Apartamento,"Barrio-Pueblo, Casco Urbano",Ponce,"Barrio-Pueblo, Casco Urbano , Ponce",https://www.clasificadosonline.com/UDRealEstateDetail.asp?ID=4838666,4838666,No Broker,https://imgcache.clasificadosonline.com//FF/FS/2025/1/13/01132025164757y1omft3z.jpg,3,1,Barrio-Pueblo,"Casco Urbano , Ponce",south
46,FINCA DE 407 CUERDAS EN CAMUY,,2600000.0,Finca,Barrio-Abra Honda,Camuy,"Barrio-Abra Honda , Camuy",https://www.clasificadosonline.com/UDRealEstateDetail.asp?ID=4838667,4838667,No Broker,https://imgcache.clasificadosonline.com//FF/FS/2025/1/13/011320251651104tuv0j0u.jpg,0,0,Barrio-Abra Honda,Camuy,north
80,Casa multifamiliar con guarderia para...,4 Cuartos | 3 Baños,290000.0,MultiFamiliar,Barrio-Coco Nuevo,Salinas,"Barrio-Coco Nuevo , Salinas",https://www.clasificadosonline.com/UDRealEstateDetail.asp?ID=4822064,4822064,No Broker,https://imgcache.clasificadosonline.com//FF/FS/2024/9/4/090420242045122fmgryrm.jpeg,4,3,Barrio-Coco Nuevo,Salinas,south


In [ ]:
cleaned_df[cleaned_df.duplicated(subset=['propertyid'], keep=False)]

In [98]:
cleaned_df

,title,price,type,barrio,pueblo,link,propertyid,broker,piclink,bedrooms,bathrooms,region
0,Beautiful Commercial Space - High Flo...,1900000.0,Comercial,Condominio-Firstbank Building,San Juan - Santurce,https://www.clasificadosonline.com/UDRealEstateDetail.asp?ID=4744137,4744137,Firstbank Building de Elite Realty LLC,https://imgcache.clasificadosonline.com//PP/FS/2023/5/20/05202023121116kjtngwcx.jpg,1,2,metro
1,"Solar, Piedras Blancas",69000.0,Solar,Barrio-Piedras Blancas,Guaynabo,https://www.clasificadosonline.com/UDRealEstateDetail.asp?ID=4827454,4827454,Piedras Blancas de GOLDEN KEY PROPERTIES PR,https://imgcache.clasificadosonline.com//PP/FS/2024/10/14/10142024103507hr5zgrwq.jpg,0,0,metro
2,RENTAS 15 MIL. 4 locales 3222 p2,625000.0,Comercial,Barrio-Certenejas,Cidra,https://www.clasificadosonline.com/UDRealEstateDetail.asp?ID=4776451,4776451,Certenejas de 1RUIZ REALTY GROUP,https://imgcache.clasificadosonline.com//PP/FS/2024/7/26/07262024191618dp5a0arp.jpeg,>=6,5,east
3,COSTAMAR BEACH VILLAGE NUEVO EN EL ME...,220000.0,Apt/WalkUp,Condominio-Costamar Beach Village,Loíza,https://www.clasificadosonline.com/UDRealEstateDetail.asp?ID=4836348,4836348,Costamar Beach Village de Marina Real Estate,https://imgcache.clasificadosonline.com//PP/FS/2024/12/19/12192024191719qtaee5kn.jpeg,2,3,metro
4,Barrio Polvorín,28000.0,Casa,Barrio-Polvorin,Cayey,https://www.clasificadosonline.com/UDRealEstateDetail.asp?ID=4838674,4838674,No Broker,https://imgcache.clasificadosonline.com//FF/FS/2025/1/13/01132025191719ucohxj3m.jpg,3,1,east
5,"VIEQUES!! Villa Borinquen - 2,818 m2 ...",79000.0,Solar,Sector-Villa Borinquen,Vieques,https://www.clasificadosonline.com/UDRealEstateDetail.asp?ID=4337287,4337287,Villa Borinquen de Spanish V.I. Properties,https://imgcache.clasificadosonline.com//PP/FS/2019/1/14/114201914059PMxzsk3y5r.jpeg,0,0,east
6,Local Comercial en el Corazon de Isla...,425000.0,Comercial,Condominio-Tropicana,Carolina - Isla Verde,https://www.clasificadosonline.com/UDRealEstateDetail.asp?ID=4822468,4822468,Tropicana de HEYLIGER REAL ESTATE,https://imgcache.clasificadosonline.com//PP/FS/2024/9/13/09132024134409t3m3k3rn.jpg,5,2,metro
7,LA CASA DE TUS SUEÑOS,695000.0,Casa,Urbanizacion-Extension Quintas De Monserrate,Ponce,https://www.clasificadosonline.com/UDRealEstateDetail.asp?ID=4830423,4830423,Extension Quintas De Monserrat de TRANS GLOBAL REALTY GROUP,https://imgcache.clasificadosonline.com//PP/FS/2025/1/6/01062025110538pjbqcv0d.jpeg,4,3,south
8,FINCA 28 CUERDAS- BARRIO ARROZAL y MI...,2500000.0,Finca,Barrio-Miraflores,Arecibo,https://www.clasificadosonline.com/UDRealEstateDetail.asp?ID=4826376,4826376,Miraflores de E. F. & ASSOCIATES REAL ESTATE,https://imgcache.clasificadosonline.com//PP/FS/2024/10/3/10032024145524qva5c2ry.jpg,0,0,north
9,URB. SANS SOUCI COURT / NUEVA EN EL M...,255000.0,Casa,Urbanizacion-Sans Souci Court,Bayamón,https://www.clasificadosonline.com/UDRealEstateDetail.asp?ID=4829292,4829292,"Sans Souci Court de DANNY FELICIANO REAL ESTATE, L",https://imgcache.clasificadosonline.com//PP/FS/2024/10/22/10222024221224kf42srnv.jpg,3,2,metro
